# Regresión logística

El modelo de regresión logística estima la probabilidad de una clase dado un vector de rasgos de la forma:

$$p(y_i|x) = \frac{exp\{w_i x + b_i\}}{\sum_j exp\{w_j x + b_j\}}$$

A partir de aquí se puede predecir una clase que clasifique al vector de entrada. Presentamos una implementación sencilla que utiliza el algoritmo de gradiente descendiente para aprender los pesos.

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

## Preparación de los datos

In [2]:
data = pd.read_csv('cat_data.csv')
data

,¿es animal?,¿es mamífero?,¿es felino?,¿es doméstico?,¿tiene dos orejas?,¿es negro?,¿tiene cuatro patas?,¿es gato?
0,1,1,1,1,1,1,1,1
1,0,0,0,1,0,1,0,0
2,1,0,1,1,0,1,1,0
3,1,1,0,1,1,0,1,0
4,1,1,1,0,1,0,1,0
5,1,1,1,1,0,0,0,1
6,1,0,0,1,1,1,0,0
7,1,1,1,1,0,0,1,1
8,1,0,0,1,0,0,0,0
9,0,0,0,0,0,0,0,0


In [3]:
#Convertir los datos a numpy
npData = data.to_numpy()
#Ejemplos
X = npData[:,:-1]
#Clases de los ejemplos
Y = npData[:,-1]

#Tamaño de los datos
#Unidades de entrada
m,d = X.shape

## Aprendizaje por GD

Determinación de hiperparámetros:

In [4]:
#Número de itraciones
its = 100
#Rango de aprendizaje
lr = 0.2

#### Entrenamiento de la red

Determinaremos los pesos a partir del algoritmo de gradiente descendiente:

$$\theta_i \leftarrow \theta_i - \eta \nabla_iR(\theta)$$

En este caso, la función de riesgo es:

$$R(\theta) = -\sum_x \sum_y y \log p(Y=y|x)$$

Donde $y$ es la clase de los datos. Además, la probabilidad se determinará por la función Softmax y $\theta = \{w,b: w \in \mathbb{R}^d, b \in \mathbb{R}\}$. Por tanto:

$$\nabla_iR(\theta) = (p(Y=y|x) - y_x) \cdot x_i$$

In [5]:
%%time

#Inicialización de los parámetros
np.random.seed(0)
w = np.random.rand(d,2)/np.sqrt(d)
b = np.ones(2)

#Detenerse
stop = False
t = 0
while  stop == False:
    #FORWARD
    #Funcion de preactivacióm
    a = np.dot(X,w)+b
    #Función de activación
    exp = np.exp(a-np.max(a))
    pred = exp/exp.sum(1,keepdims=True)

    #Error
    pred[range(m),Y] -= Y
    #Derivada
    DW = np.dot(X.T,pred)
    Db = pred.sum(0)
    
    #ACTUALIZACIÓN
    #Gradiente descendiente
    w -= lr*DW + 0.001*DW
    b -= lr*Db
    
    t += 1
    #Criterio de paro
    if t > its: 
        stop = True

CPU times: user 17.6 ms, sys: 4.91 ms, total: 22.5 ms
Wall time: 20.1 ms


## Aplicación de regrsión logística

Finalmente, una vez obtenido los pesos, podemos estimar las probabilidades de los objetos que estamos trabajando. En este caso, se estima la probabilidad para las dos clases 0 y 1.

También podemos obtener una clasificación basada en maximizar la probabilidad:

$$\hat{y} = \arg\max_y p(Y=y|x)$$

In [6]:
#Predicción
def estimate(X):
    #Pre-activación
    a = np.dot(X,w)+b
    #Activación
    exp = np.exp(a)
    f = exp/exp.sum(1,keepdims=True)
    #Clasificación
    cl = np.argmax(f, axis=1)
    
    return cl, f

In [7]:
clases, probs = estimate(X)
print('Clases predichas: {} \nClases esperadas: {} \nError: {}'.format(clases,Y, np.abs(clases-Y).sum(0)/len(Y)))

Clases predichas: [0 0 0 0 0 1 0 1 0 0 0 0 1 1] 
Clases esperadas: [1 0 0 0 0 1 0 1 0 0 0 0 1 1] 
Error: 0.07142857142857142


#### Exploración de los pesos

El perceptrón que hemos definido, determina las probabilidades para la clase 1 (gato) y la clase 0 (no gato) y elige la clase que maximice la probabilidad.

Las probabilidades de salida son:

In [8]:
datos = pd.DataFrame(data=probs, columns=['Prob. clase 0', 'Prob. clase 1'])
datos['Clase real'] = Y
datos['Clase predica'] = clases
datos

,Prob. clase 0,Prob. clase 1,Clase real,Clase predica
0,0.577072,0.422928,1,0
1,0.575415,0.424585,0,0
2,0.940000,0.060000,0,0
3,0.928751,0.071249,0,0
4,0.874771,0.125229,0,0
5,0.006457,0.993543,1,1
6,0.970370,0.029630,0,0
7,0.167570,0.832430,1,1
8,0.828522,0.171478,0,0
9,0.874026,0.125974,0,0


Los pesos de las conexiones que ha aprendido para la clase 1 (gato) son los siguientes:

In [9]:
pd.DataFrame(data=np.append(w[:,1],b[1]), index=list(data.columns)[:7]+['bias'], columns=['Pesos de clase gato'])

,Pesos de clase gato
¿es animal?,-72.241816
¿es mamífero?,-38.999461
¿es felino?,-38.479844
¿es doméstico?,-48.893160
¿tiene dos orejas?,-50.663191
¿es negro?,-41.144012
¿tiene cuatro patas?,-52.186844
bias,-90.868519
